https://github.com/onnx/tutorials/blob/master/tutorials/TensorflowToOnnx-1.ipynb

# Convert Tensorflow model to ONNX
Tensorflow and ONNX both define their own graph format to represent to model. You can use [tensorflow-onnx](https://github.com/onnx/tensorflow-onnx "Title") to export a Tensorflow model to ONNX.

We divide the guide into 2 parts: part 1 covers basic conversion and part 2 advanced topics. The following content will be covered in order:
1. Procedures to convert tensorflow model
   - get tensorflow model
   - convert to ONNX
   - validate
2. Key conceptions
   - opset
   - data format

## Step 1 - Get Tensorflow model
Tensorflow uses several file formats to represent a model, such as checkpoint files, graph with weight(called `frozen graph` next) and saved_model, and it has APIs to generate these files, you can find the code snippets in the script [tensorflow_to_onnx_example.py](./assets/tensorflow_to_onnx_example.py)

And `tensorflow-onnx` can accept all the three formats to represent a Tensorflow model, **the format "saved_model" should be the preference** since it doesn't require the user to specify input and output names of graph.
we will cover it in this section and cover the other two in the last section. And also, you could get more detail from `tensorflow-onnx`'s [README](https://github.com/onnx/tensorflow-onnx/blob/master/README.md "Title") file.

In [2]:
import tensorflow_datasets as tfds
mnist = tfds.load(name='mnist')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to C:\Users\weldl\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [1]:
import os
import shutil
import tensorflow as tf
from assets.tensorflow_to_onnx_example import create_and_train_mnist
def save_model_to_saved_model(sess, input_tensor, output_tensor):
    from tensorflow.saved_model import simple_save
    save_path = r"./output/saved_model"
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    simple_save(sess, save_path, {input_tensor.name: input_tensor}, {output_tensor.name: output_tensor})

print("please wait for a while, because the script will train MNIST from scratch")
tf.reset_default_graph()
sess_tf, saver, input_tensor, output_tensor = create_and_train_mnist()
print("save tensorflow in format \"saved_model\"")
save_model_to_saved_model(sess_tf, input_tensor, output_tensor)

ModuleNotFoundError: No module named 'input_data'

## Step 2 - Convert to ONNX
`tensorflow-onnx` has several entries to convert tensorflow model with different tensorflow formats, this section will cover "saved_model" only, "frozen graph" and "checkpoint" will be covered in [part 2](./TensorflowToOnnx-2.ipynb).

Also, `tensorflow-onnx` has exported related python APIs, so users can call them directly on their script instead of command line, also the detail will be covered in [part 2](./TensorflowToOnnx-2.ipynb).

In [13]:
# generating mnist.onnx using saved_model
!python -m tf2onnx.convert \
        --saved-model ./output/saved_model \
        --output ./output/mnist1.onnx \
        --opset 7

2019-06-17 07:22:03,871 - INFO - Using tensorflow=1.12.0, onnx=1.5.0, tf2onnx=1.5.1/0c735a
2019-06-17 07:22:03,871 - INFO - Using opset <onnx, 7>
2019-06-17 07:22:03,989 - INFO - 
2019-06-17 07:22:04,012 - INFO - Optimizing ONNX model
2019-06-17 07:22:04,029 - INFO - After optimization: Add -2 (4->2), Identity -3 (3->0), Transpose -8 (9->1)
2019-06-17 07:22:04,031 - INFO - 
2019-06-17 07:22:04,032 - INFO - Successfully converted TensorFlow model ./output/saved_model to ONNX
2019-06-17 07:22:04,044 - INFO - ONNX model is saved at ./output/mnist1.onnx


## Step 3 - Validate
There are several framework can run model in ONNX format, here [ONNXRuntime](https://github.com/microsoft/onnxruntime "Title") , opensourced by `Microsoft`, is used to make sure the generated ONNX graph behaves well.
The input "image.npz" is an image of handwritten "7", so the expected classification result of model should be "7". 

In [14]:
import numpy as np
import onnxruntime as ort

img = np.load("./assets/image.npz").reshape([1, 784])  
sess_ort = ort.InferenceSession("./output/mnist1.onnx")
res = sess_ort.run(output_names=[output_tensor.name], input_feed={input_tensor.name: img})
print("the expected result is \"7\"")
print("the digit is classified as \"%s\" in ONNXRruntime"%np.argmax(res))

the expected result is "7"
the digit is classified as "7" in ONNXRruntime


## Key conceptions
This command line should work for most tensorflow models if they are available a saved_model. In some cases you might encounter issues that require extra options.

The most important concept is "**opset** version": ONNX is an evolving standard, for example it will add more new operations and enhance existing operations, so different opset version will contain different operations and operations may have different behavior. The default version "tensorflow-onnx" used is 7 and ONNX supports version 10 now, so if the conversion failed, you may try different version, by command line option "--opset", to see if it works.

Continue with [part 2](./TensorflowToOnnx-2.ipynb) that explains advanced topics.